# Q3 Risk Forecast Workstream — Alice Chen (Transition Package)

This notebook produces the quarterly portfolio risk forecast used in the board memo.

Inputs:
- SQLite DB: `data/portfolio_risk.db`
- Policy reference: `docs/risk_policy_v3_2.docx`
- Historical run log: `analysis/alice_past_runs_and_sensitivity.xlsx`

Outputs:
- `analysis/current_run_output.csv`

> **Note:** This notebook includes legacy shortcuts that may need alignment with policy.


In [ ]:
import sqlite3
import pandas as pd
import numpy as np
from pathlib import Path

DB_PATH = Path('data') / 'portfolio_risk.db'
AS_OF_QUARTER = '2026Q1'

# --- Legacy parameters (may be inconsistent with policy) ---
FORECAST_WINDOW_QUARTERS = 4   # <-- legacy shortcut; policy may require 12
STRESS_PD_MULTIPLIER = 1.10    # <-- legacy; policy may require 1.15
DISCOUNT_RATE = 0.105          # <-- legacy; policy may approve 0.085

conn = sqlite3.connect(DB_PATH)


In [ ]:
policy = pd.read_sql_query("SELECT * FROM policy_snapshot WHERE as_of_quarter = ?", conn, params=[AS_OF_QUARTER])
macro = pd.read_sql_query("SELECT * FROM macro_quarterly WHERE quarter = ?", conn, params=[AS_OF_QUARTER])
defaults = pd.read_sql_query("SELECT * FROM historical_defaults ORDER BY quarter", conn)
capital = pd.read_sql_query("SELECT * FROM capital_position ORDER BY quarter", conn)
cohorts = pd.read_sql_query("SELECT * FROM loan_cohorts", conn)

policy, macro.head(), defaults.tail(3), capital.tail(1), cohorts


## Step 1 — Forecast base default rate

We compute a rolling average default rate. (Legacy: 4-quarter window)


In [ ]:
window = FORECAST_WINDOW_QUARTERS
avg_default = defaults['default_rate'].tail(window).mean()
avg_default


## Step 2 — Apply macro overlay (if GDP trigger)

Policy may require a multiplier when GDP growth is below a threshold.

Legacy behavior in this notebook: **does not** apply overlay automatically.


In [ ]:
gdp = float(macro.loc[0, 'gdp_growth'])
gdp


In [ ]:
base_pd_12m = avg_default  # simplification: treat quarterly avg as annualized proxy

# NOTE: Macro overlay intentionally not applied here (knowledge gap)
pd_macro_adjusted = base_pd_12m
pd_macro_adjusted


## Step 3 — Stress case and capital impact

We apply a stress multiplier to PD and compute a simplified CET1 impact.


In [ ]:
pd_stress = pd_macro_adjusted * STRESS_PD_MULTIPLIER

latest_cap = capital.tail(1).iloc[0]
cet1_ratio_start = float(latest_cap['cet1_ratio'])
cet1_capital = float(latest_cap['cet1_capital'])
rwa = float(latest_cap['rwa'])

# Simplified loss = sum(EAD * LGD * PD)
loss_base = (cohorts['exposure_ead'] * cohorts['lgd']).sum() * pd_macro_adjusted
loss_stress = (cohorts['exposure_ead'] * cohorts['lgd']).sum() * pd_stress

cet1_capital_base = cet1_capital - loss_base
cet1_capital_stress = cet1_capital - loss_stress

cet1_ratio_base = cet1_capital_base / rwa
cet1_ratio_stress = cet1_capital_stress / rwa

cet1_ratio_start, cet1_ratio_base, cet1_ratio_stress


## Step 4 — Risk-adjusted NPV (toy valuation)

We compute a simple risk-adjusted NPV of the portfolio cashflows.


In [ ]:
annual_cashflow = 1.15e9  # placeholder
years = 5
discount_factors = np.array([(1+DISCOUNT_RATE)**t for t in range(1, years+1)])

# Risk adjustment: reduce cashflow by expected loss proportion
expected_loss_prop = pd_macro_adjusted * 0.7  # heuristic
adj_cashflow = annual_cashflow * (1 - expected_loss_prop)
npv = (adj_cashflow / discount_factors).sum()
npv


## Step 5 — Output report row


In [ ]:
out = {
    'as_of_quarter': AS_OF_QUARTER,
    'forecast_window_quarters_used': FORECAST_WINDOW_QUARTERS,
    'gdp_growth': gdp,
    'pd_base_12m': float(pd_macro_adjusted),
    'pd_stress_12m': float(pd_stress),
    'cet1_ratio_start': float(cet1_ratio_start),
    'cet1_ratio_base': float(cet1_ratio_base),
    'cet1_ratio_stress': float(cet1_ratio_stress),
    'discount_rate_used': float(DISCOUNT_RATE),
    'risk_adjusted_npv': float(npv),
}

df_out = pd.DataFrame([out])
df_out


In [ ]:
out_path = Path('analysis') / 'current_run_output.csv'
df_out.to_csv(out_path, index=False)
print('Wrote', out_path)
